# Check the implementation from:
https://geemap.org/notebooks/31_unsupervised_classification/ 

In [1]:
import gc
gc.collect()

10

In [2]:
import geemap
import ee

In [3]:
Map = geemap.Map()
Map.add_basemap("SATELLITE")
# Map

## Change to Sentinel 2 TOA and check cloud covers; change ROI and clustering ROI for better accuracy!

In [4]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799]) # Michigan
# point = ee.Geometry.Point([18.08, 59.32]) # Stockholm
# region = ee.Geometry.BBox(17.82, 59.22, 18.34, 59.42) # Stockholm
region = ee.Geometry.BBox(17.70, 59.20, 18.70, 59.60) # Stockholm

image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(region)
    .filterDate('2023-06-01', '2023-08-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .mosaic()
    .unitScale(0, 10000)
    .clip(region) # Don't clip as we want to analyse the whole image
    # .select('B[1-7]')
)

# True Color Display:
vis_params = {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(region, 8)
Map.addLayer(image, vis_params, "Sentinel-2")

In [5]:
props = geemap.image_props(image)
props.getInfo()
# props.get('IMAGE_DATE').getInfo() # For LandSat-8; Sentinel-2 does not have these image properties
# props.get('CLOUD_COVER').getInfo() # For LandSat-8; Sentinel-2 does not have these image properties

{'NOMINAL_SCALE': 111319.49079327357,
 'system:band_names': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B8A',
  'B9',
  'B11',
  'B12',
  'AOT',
  'WVP',
  'SCL',
  'TCI_R',
  'TCI_G',
  'TCI_B',
  'MSK_CLDPRB',
  'MSK_SNWPRB',
  'QA10',
  'QA20',
  'QA60']}

# Construct Training Data Set:

In [6]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')
input = image.addBands([ndvi, ndwi])

# Define the input features for the classifier
# inputFeatures = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'ndvi', 'ndwi']
inputFeatures = ['B2', 'B3', 'B4', 'B8', 'B11', 'ndvi', 'ndwi']

In [7]:
# Make the training dataset.
training = input.select(inputFeatures).sample(
    **{
        'region': region,
        'scale': 30,
        'numPixels': 5000, # Default, given
        # 'numPixels': 10000,
        # 'seed': 0,
        # 'geometries': True,  # Set this to False to ignore geometries
    }
)

# Plot the training sample data (not really necessary)
# Map.addLayer(training, {}, 'training', False)
# Map

In [8]:
# Instantiate the clusterer and train it.
n_clusters = 15
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [9]:
# Cluster the input using the trained clusterer.
result_unclassified = input.select(inputFeatures).cluster(clusterer)
# geemap.image_props(result)

# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'clusters')
# Map.addLayer(result, {}, 'clusters')
# Map

In [10]:
# Define 15 colors to distinguish the clusters and look at what they are!
# legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
# legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
legend_keys = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']
legend_colors = ['#e6194B','#3cb44b','#ffe119','#4363d8','#f58231','#911eb4','#42d4f4','#f032e6','#bfef45','#fabed4','#469990','#9A6324','#800000','#808000','#000075']

Map.addLayer(
    result_unclassified, {'min': 0, 'max': 14, 'palette': legend_colors}, 'Unclassified clusters'
)
Map.add_legend(
    keys=legend_keys, colors=legend_colors, position='bottomright'
)
Map

Map(center=[59.399925146053874, 18.19999999999986], controls=(WidgetControl(options=['position', 'transparent_…

Water: 1 (high shallow), 2 (deep), 8 (mid shallow), 13 (shallow), 14 (?, close to outer seas)
Vegetation: 0 (larger), 3, 4 (slight veg) , 5 (tall), 11 (more disperse veg)
Urban: 7 (darker/road), 9 (metallic rooftop or cloud)
Bare Field: 6 (a bit more veg), 10 (barer), 12 (even more barer)

In [11]:
# Define Classification Correspondance Scheme: ['Water', 'Vegetation', 'Urban', 'Bare Field'] = [0,1,2,3]
legend_keys_classified = ['Water', 'Vegetation', 'Urban', 'Bare Field']
legend_colors_classified = ['#4363d8', '#3cb44b', '#a9a9a9', '#ffe119']

# Remap the spectral classes from clusters:
fromClusters = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
toClasses = [1, 1, 0, 1, 1, 1, 3, 2, 1, 2, 3, 1, 3, 0, 0]

# Reclassify the map
result_classified = result_unclassified.remap(fromClusters, toClasses)

Map.addLayer(
    result_classified, {'min': 0, 'max': 3, 'palette': legend_colors_classified}, 'Labelled clusters'
)
Map.add_legend(
    keys=legend_keys_classified, colors=legend_colors_classified, position='bottomleft'
)
Map

Map(center=[59.399925146053874, 18.19999999999986], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

# Image Export (optional)
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

geemap.ee_export_image(result, filename=out_file, scale=90)
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)

# Split bar (problematic, to remove)
# import geemap.foliumap as gf
import geemap as gf
Map2 = gf.Map()
Map2.centerObject(region, 8)
Map2.add_basemap("SATELLITE")

tile_img = gf.ee_tile_layer(image, vis_params, 'Sentinel-2 True Colour')
tile_cluster = gf.ee_tile_layer(result, {'min': 0, 'max': 14, 'palette': legend_colors}, 'Sentinel-2 Clustered Unclassified')


Map2.split_map(left_layer=tile_img, right_layer=tile_cluster)
Map2.addLayerControl

Map2.add_legend(
    labels=legend_keys, colors=legend_colors, position='bottomright'
)
# Map2.add_legend(
#     keys=legend_keys, colors=legend_colors, position='bottomright'
# )

Map2

# Unsupervised Classification - Accuracy Assessment:

In [13]:
# Import User Labelled Polygons from Supervised Classification for Accuracy Assessment:
# Class 0 - Water:
water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.1287605285643, 59.420668377041466],
                  [18.1287605285643, 59.41916191061126],
                  [18.132365417480315, 59.41916191061126],
                  [18.132365417480315, 59.420668377041466]]], None, False),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.155077293266338, 59.390922117625365],
                  [18.155077293266338, 59.3900917489387],
                  [18.15679390703587, 59.3900917489387],
                  [18.15679390703587, 59.390922117625365]]], None, False),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.103407218803447, 59.36862621806919],
                  [18.103407218803447, 59.36823262938389],
                  [18.104523017753642, 59.36823262938389],
                  [18.104523017753642, 59.36862621806919]]], None, False),
            {
              "landcover": 0,
              "system:index": "2"
            })])
# Class 1 - Vegetation:
vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01369561111225, 59.34494428938679],
                  [18.01369561111225, 59.344594188824566],
                  [18.01425351058735, 59.344594188824566],
                  [18.01425351058735, 59.34494428938679]]], None, False),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.014038933866157, 59.341924553350076],
                  [18.014038933866157, 59.341661954918315],
                  [18.0149830714394, 59.341661954918315],
                  [18.0149830714394, 59.341924553350076]]], None, False),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.02884909415171, 59.36172374248307],
                  [18.02884909415171, 59.36128633246986],
                  [18.030436961888526, 59.36128633246986],
                  [18.030436961888526, 59.36172374248307]]], None, False),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.060434787511085, 59.3606520779365],
                  [18.060434787511085, 59.35982096784676],
                  [18.061378925084327, 59.35982096784676],
                  [18.061378925084327, 59.3606520779365]]], None, False),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.064468829869483, 59.363320241188205],
                  [18.064468829869483, 59.362707894399826],
                  [18.065370052098487, 59.362707894399826],
                  [18.065370052098487, 59.363320241188205]]], None, False),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.05060717368052, 59.36286098213282],
                  [18.05060717368052, 59.36255480597621],
                  [18.051122157811378, 59.36255480597621],
                  [18.051122157811378, 59.36286098213282]]], None, False),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.071378200291846, 59.35533699654806],
                  [18.071378200291846, 59.3545713812703],
                  [18.07257982993052, 59.3545713812703],
                  [18.07257982993052, 59.35533699654806]]], None, False),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.07506891989634, 59.355555740598646],
                  [18.07506891989634, 59.35481200507544],
                  [18.075712650059913, 59.35481200507544],
                  [18.075712650059913, 59.355555740598646]]], None, False),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.080274348227597, 59.364458675875035],
                  [18.080274348227597, 59.36371513542555],
                  [18.081046824423886, 59.36371513542555],
                  [18.081046824423886, 59.364458675875035]]], None, False),
            {
              "landcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.083793406455136, 59.36660173071935],
                  [18.083793406455136, 59.36585823722938],
                  [18.08473754402838, 59.36585823722938],
                  [18.08473754402838, 59.36660173071935]]], None, False),
            {
              "landcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.091518168418027, 59.36375887355005],
                  [18.091518168418027, 59.36314653467683],
                  [18.09194732186041, 59.36314653467683],
                  [18.09194732186041, 59.36375887355005]]], None, False),
            {
              "landcover": 1,
              "system:index": "10"
            })])
# Class 2 - Urban:
urban = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.053864163954678, 59.332220234026025],
                  [18.053864163954678, 59.33169488507395],
                  [18.05482975920004, 59.33169488507395],
                  [18.05482975920004, 59.332220234026025]]], None, False),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.126311076754277, 59.3412618738503],
                  [18.126311076754277, 59.34058347717914],
                  [18.127383960360234, 59.34058347717914],
                  [18.127383960360234, 59.3412618738503]]], None, False),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08801953187299, 59.30215086123935],
                  [18.08801953187299, 59.30188795534564],
                  [18.088427227643255, 59.30188795534564],
                  [18.088427227643255, 59.30215086123935]]], None, False),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.091216725018743, 59.304976971271074],
                  [18.091216725018743, 59.30459359801732],
                  [18.091667336133245, 59.30459359801732],
                  [18.091667336133245, 59.304976971271074]]], None, False),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.098877113965276, 59.30182222855471],
                  [18.098877113965276, 59.30118686302864],
                  [18.09977833619428, 59.30118686302864],
                  [18.09977833619428, 59.30182222855471]]], None, False),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.110163849499944, 59.308306660060424],
                  [18.110163849499944, 59.307978086848095],
                  [18.110936325696233, 59.307978086848095],
                  [18.110936325696233, 59.308306660060424]]], None, False),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.066390198376897, 59.307101876095324],
                  [18.066390198376897, 59.30686091418073],
                  [18.06681935181928, 59.30686091418073],
                  [18.06681935181928, 59.307101876095324]]], None, False),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.054288071301702, 59.30118686302864],
                  [18.054288071301702, 59.301011587691676],
                  [18.05463139405561, 59.301011587691676],
                  [18.05463139405561, 59.30118686302864]]], None, False),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.075874489453557, 59.306357079206215],
                  [18.075874489453557, 59.30611611201477],
                  [18.076260727551702, 59.30611611201477],
                  [18.076260727551702, 59.306357079206215]]], None, False),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08102433076215, 59.29939024801239],
                  [18.08102433076215, 59.29859050471372],
                  [18.081496399548772, 59.29859050471372],
                  [18.081496399548772, 59.29939024801239]]], None, False),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01549805557026, 59.34995470818497],
                  [18.01549805557026, 59.349123336140345],
                  [18.01674260055317, 59.349123336140345],
                  [18.01674260055317, 59.34995470818497]]], None, False),
            {
              "landcover": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.029145135038032, 59.34636653580566],
                  [18.029145135038032, 59.3458632861902],
                  [18.030132187955513, 59.3458632861902],
                  [18.030132187955513, 59.34636653580566]]], None, False),
            {
              "landcover": 2,
              "system:index": "11"
            })])
# Class 3 - Bare Field
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[17.983144676571825, 59.48949503893704],
                  [17.983144676571825, 59.48892854715184],
                  [17.983960068112353, 59.48892854715184],
                  [17.983960068112353, 59.48949503893704]]], None, False),
            {
              "landcover": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.999624168759325, 59.48749048685693],
                  [17.999624168759325, 59.48659711552939],
                  [18.00129786718462, 59.48659711552939],
                  [18.00129786718462, 59.48749048685693]]], None, False),
            {
              "landcover": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.002842819577197, 59.49544262887063],
                  [18.002842819577197, 59.494898021360086],
                  [18.003701126461962, 59.494898021360086],
                  [18.003701126461962, 59.49544262887063]]], None, False),
            {
              "landcover": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04191724050615, 59.494505898511925],
                  [18.04191724050615, 59.49410287861594],
                  [18.042582428341845, 59.49410287861594],
                  [18.042582428341845, 59.494505898511925]]], None, False),
            {
              "landcover": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.88718596685503, 59.49958135881831],
                  [17.88718596685503, 59.49840513982099],
                  [17.889331734066943, 59.49840513982099],
                  [17.889331734066943, 59.49958135881831]]], None, False),
            {
              "landcover": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.887774834533488, 59.591804275916374],
                  [17.887774834533488, 59.59067471774544],
                  [17.88949144830302, 59.59067471774544],
                  [17.88949144830302, 59.591804275916374]]], None, False),
            {
              "landcover": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.90395391931132, 59.58652543804897],
                  [17.90395391931132, 59.58411384133924],
                  [17.905756363769328, 59.58411384133924],
                  [17.905756363769328, 59.58652543804897]]], None, False),
            {
              "landcover": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.104920177676632, 59.342211009243925],
                  [18.104920177676632, 59.341871822081195],
                  [18.105413704135373, 59.341871822081195],
                  [18.105413704135373, 59.342211009243925]]], None, False),
            {
              "landcover": 3,
              "system:index": "7"
            })])

Map.addLayer(water, {}, "water")
Map.addLayer(vegetation, {}, "vegetation")
Map.addLayer(urban, {}, "urban")
Map.addLayer(bare, {}, "bare")
Map

Map(bottom=19548.0, center=[59.399925146053874, 18.19999999999986], controls=(WidgetControl(options=['position…

In [14]:
# Combine labelled polygons to one training feature:
trainingFeatures = water.merge(vegetation).merge(urban).merge(bare)
classifierTraining = input.select(inputFeatures).sampleRegions(
  **{
  'collection': trainingFeatures,
  'properties': ['landcover'],
  'scale': 30,
  'geometries': True
  }
)

# Separate into training & testing sets, but only the testing set is used for Unsupervised Learning Accuracy Assessment:
trainingTesting = classifierTraining.randomColumn()
trainingSet = trainingTesting.filter(ee.Filter.lessThan('random',0.6))
testingSet = trainingTesting.filter(ee.Filter.greaterThanOrEquals('random',0.6))

In [15]:
result_classified.bandNames().get(0) # Verify the property inherited from the remap function

In [16]:
# Construct Confusion Matrix:
testVec = result_classified.sampleRegions(
    **{
        'collection': testingSet,
        'scale': 30    
    }
)

confusionMatrix = testVec.errorMatrix('landcover',result_classified.bandNames().get(0))

# Print Confusion Matrix and relevant accuracies:

In [17]:
confusionMatrix

In [18]:
confusionMatrix.accuracy()

In [19]:
confusionMatrix.producersAccuracy()

In [20]:
confusionMatrix.consumersAccuracy()